In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
# !pip install faiss-cpu
# !pip install deepface==0.0.92
# !pip install tensorflow==2.12.0
# !pip install keras==2.12.0
# # !pip install facenet-pytorch

In [ ]:
!pip install ranx
from ranx import Run, fuse

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.4 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=340f201d46af19edc4a19fda1f6a38de7d6ff2e3f7d1025d83718be776186aa5
  Stored in directory: /root/.cache/pip/wheels/63/f9/dc/2dd16d3330e327236e4d407941975c42d5159d200cdb7922d8
  Created wheel for cbor: filename=cbor-1.0.0-cp311-cp311-linux_x86_64.whl size=53930 sha256=f1058361ce6a7e537e7f94001f244b2439ffdf2fef182bdf956404b7872bad2d
 

In [ ]:
import torch
from transformers import BeitFeatureExtractor, BeitModel
from deepface import DeepFace
from PIL import Image
import os
import pandas as pd
import numpy as np
import faiss
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import auc
from IPython.display import clear_output
import itertools

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
import clip

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-b2zsn6u8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-b2zsn6u8
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
#Tạo model xử lý ảnh clip
class ImageEmbedding():
    def __init__(self, backbone="ViT-B/32"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load(backbone, device=self.device)
        # self.model, self.preprocess = clip.load('RN101', device=self.device)

    def __call__(self, image):
        # image = Image.open(image_path).convert("RGB")
        image_input = self.preprocess(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            image_feature = self.model.encode_image(image_input)[0]

        return image_feature.detach().cpu().numpy()

In [ ]:
def precision(true_positives, retrieved):
  return true_positives / retrieved if retrieved != 0 else 0

def recall(true_positives, relevant):
  return true_positives / relevant if relevant != 0 else 0

def f1_score(precision, recall):
  return 2 * precision * recall / (precision + recall)

def average_precision(relevant_items, retrieved_items):
    '''
    relevant_items: list các ID mà thực sự là đúng
    retrieved_items: list các ID được truy vấn trả về
    '''
    relevant_items = set(relevant_items)
    true_positives = 0
    ap = 0

    for i, item in enumerate(retrieved_items):
        if item in relevant_items:
            true_positives += 1
            ap += true_positives / (i + 1)

    return ap / len(relevant_items) if relevant_items else 0


def mean_average_precision(queries):
  aps = [average_precision(q[0], q[1]) for q in queries]
  return np.mean(aps) if aps else 0

# Hàm vẽ Precision-Recall Curve
def plot_precision_recall_curve(queries, tittle='Precision-Recall Curve'):
    plt.figure(figsize=(10, 6))

    for idx, (relevant_items, retrieved_items) in enumerate(queries):
        relevant_items = set(relevant_items)
        precision_vals = []
        recall_vals = []
        true_positives = 0

        # Tính Precision và Recall cho từng ngưỡng
        for i, item in enumerate(retrieved_items):
            if item in relevant_items:
                true_positives += 1
            p = precision(true_positives, i + 1)
            r = recall(true_positives, len(relevant_items))
            precision_vals.append(p)
            recall_vals.append(r)

        auc_score = auc(recall_vals, precision_vals)

        # Vẽ đường Precision-Recall cho từng truy vấn
        plt.plot(recall_vals, precision_vals, label=f'AUC = {auc_score:.2f}')

    # Cấu hình biểu đồ
    plt.title(tittle)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()
    plt.grid()
    plt.show()

# Hàm vẽ biểu đồ AP và mAP
def plot_ap_with_map(queries):
    # Tính AP cho từng truy vấn
    aps = [average_precision(q[0], q[1]) for q in queries]
    map_score = mean_average_precision(queries)

    plt.figure(figsize=(10, 6))

    # Vẽ cột AP
    x = np.arange(len(aps))  # Vị trí các cột
    plt.bar(x, aps, color='blue', alpha=0.7, label='Average Precision (AP)')

    # Vẽ đường mAP
    plt.axhline(y=map_score, color='red', linestyle='--', label=f'mAP = {map_score:.2f}')

    # Cấu hình biểu đồ
    plt.title('Average Precision (AP) and Mean Average Precision (mAP)')
    plt.xlabel('Queries')
    plt.ylabel('Average Precision')
    plt.xticks(x, [f'Query {i+1}' for i in range(len(aps))])  # Nhãn các query
    plt.ylim(0, 1)  # AP nằm trong khoảng [0, 1]
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()


In [ ]:
def merge_sort_by_counter(results):
  merged_results = results.flatten()
  counts = Counter(merged_results)
  merged_results = sorted(counts, key=counts.get, reverse=True)
  return merged_results

In [ ]:
def merge_sort_by_distance(results, D):
  combined = []
  for query_distances, query_indices in zip(D, results):
      combined.extend(zip(query_distances, query_indices))

    # Sắp xếp dựa trên giá trị khoảng cách (D) theo thứ tự giảm dần
  combined_sorted = sorted(combined, key=lambda x: x[0], reverse=True)

  # Lọc để giữ lại giá trị có D cao nhất cho mỗi chỉ số trong I
  filtered_indices = {}
  for distance, index in combined_sorted:
      if index not in filtered_indices:  # Chỉ thêm nếu chưa tồn tại
          filtered_indices[index] = distance

  # Kết quả cuối cùng: chỉ số đã lọc
  result = list(filtered_indices.keys())

  return result

In [ ]:
def merge_sort_by_ranx(results, D, method='gmnz'):
  # Chuyển đổi dữ liệu sang dạng dictionary
  runs = []
  for i, (res, sims) in enumerate(zip(results, D)):
      query_id = f"query_0"
      run_data = {str(doc_id): score for doc_id, score in zip(res, sims)}
      run = Run({query_id: run_data})
      runs.append(run)
  # Hợp nhất kết quả
  fused_run = fuse(runs, method=method)  # Có thể điều chỉnh weights nếu cần

  fused_ranking_results = {}

  for query_id, docs in fused_run.to_dict().items():
      fused_ranking_results[query_id] = list(docs.keys())  # Lấy danh sách document IDs (không cần điểm số)

  return fused_ranking_results['query_0']

# Extract features

In [ ]:
def normalize_data(data):
    # Chuẩn hóa dữ liệu
    normalized_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    return normalized_data

In [ ]:
temp_path = "/content/drive/MyDrive/TRECVID/frames_5fps/like_me/like_me-12/like_me-12-shot_5/like_me-12-shot_5-frame_0.jpg"

In [ ]:
# BEiT
model_beit_name = "microsoft/beit-base-patch16-224-pt22k"
extractor_beit = BeitFeatureExtractor.from_pretrained(model_beit_name)
model_beit = BeitModel.from_pretrained(model_beit_name)

# Trích xuất đặc trưng theo từng batch ảnh
def extract_beit_feature(image_paths, general=False):
  images = [Image.open(path).convert('RGB') for path in image_paths]
  # Chuyển đổi ảnh thành các đầu vào phù hợp với BEiT
  inputs = extractor_beit(images=images, return_tensors="pt")

  # Lấy đặc trưng từ mô hình BEiT
  with torch.no_grad():
      outputs = model_beit(**inputs)

  if general: # sử dụng patch 0
    features = outputs.last_hidden_state[:,0,:]  # Tensor có shape (batch_size, hidden_size)
  else:
    features = outputs.last_hidden_state.mean(dim=1)  # Tensor có shape (batch_size, hidden_size)
  return features

example = extract_beit_feature([temp_path])
example.shape

preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/368M [00:00<?, ?B/s]

Some weights of BeitModel were not initialized from the model checkpoint at microsoft/beit-base-patch16-224-pt22k and are newly initialized: ['pooler.layernorm.bias', 'pooler.layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 768])

In [ ]:
clip_extractor_rn = ImageEmbedding(backbone="RN101")
clip_extractor_vit = ImageEmbedding()

def extract_clip_feature(image_paths, general=False):
  images = [Image.open(path).convert('RGB') for path in image_paths]
  features = []
  for image in images:
    if general:
      feature = clip_extractor_rn(image)
    else:
      feature = clip_extractor_vit(image)

    features.append(feature)
  features = np.array(features)
  return features

example = extract_clip_feature([temp_path])
example.shape

100%|███████████████████████████████████████| 278M/278M [00:20<00:00, 14.0MiB/s]
100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 156MiB/s]


(1, 512)

# Parameter

In [ ]:
query_path = '/content/drive/MyDrive/TRECVID/query2024'
database_path = '/content/drive/MyDrive/TRECVID/Group_Duy_Thang/Features'
project_path = '/content/drive/MyDrive/TRECVID'
ground_truth_path = '/content/drive/MyDrive/TRECVID/Group_Duy_Thang/ground-truth'
list_films = ['Calloused_Hands', 'Liberty_Kid', 'like_me', 'losing_ground', 'Memphis', 'Archipelago', 'bonneville', 'ChainedforLife', 'Heart_Machine', 'little_rock']

In [ ]:
def load_ground_truth(film_name):
  gt_path = os.path.join(ground_truth_path, f'{film_name}.xlsx')
  df = pd.read_excel(gt_path)
  result = {}
  for col in df.columns[3:]:
    # Lọc các dòng có giá trị bằng 1 ở cột hiện tại và lấy danh sách các shot tương ứng
    shots = df[df[col] == 1]['Shot'].tolist()
    result[col] = shots
  return result

In [ ]:
k = 50000
feature_with_face_options = ["Facenet", "ArcFace"]
feature_without_face_options = ['BEiT', 'CLIP']
remove_type_options = ['Compare', 'DBSCAN','']
general_options = [True, False] # true: beit_s, clip_rn
merge_options = ["Counter", "Distance", "Ranx"]

grid = list(itertools.product(feature_with_face_options, feature_without_face_options, remove_type_options, general_options, merge_options))
param_grid = [
    {
        "feature_with_face": fwf,
        "feature_without_face": fwout,
        "remove_type": rt,
        "general": gen,
        "merge_type": mer
    }
    for fwf, fwout, rt, gen, mer in grid
]

for i, config in enumerate(param_grid):
    print(f"Tổ hợp {i+1}: {config}")

Tổ hợp 1: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': True, 'merge_type': 'Counter'}
Tổ hợp 2: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': True, 'merge_type': 'Distance'}
Tổ hợp 3: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': True, 'merge_type': 'Ranx'}
Tổ hợp 4: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': False, 'merge_type': 'Counter'}
Tổ hợp 5: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': False, 'merge_type': 'Distance'}
Tổ hợp 6: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'Compare', 'general': False, 'merge_type': 'Ranx'}
Tổ hợp 7: {'feature_with_face': 'Facenet', 'feature_without_face': 'BEiT', 'remove_type': 'DBSCAN', 'general': True, 'merge_type': 'Counter'}
T

# Get input features

In [ ]:
def get_input_features(list_query, feature_with_face, feature_without_face, general):
  input_features_with_face = []
  cant_detect_face_paths = []
  for i, image_path in enumerate(list_query):
    try:
      # img = Image.open(image_path)
      # img = np.asarray(img)
      image = Image.open(image_path).convert('RGB')
      faces_features = DeepFace.represent(image_path, model_name= feature_with_face, detector_backend='mtcnn')
      # faces_features = DeepFace.represent(image_path, model_name= feature_with_face, detector_backend='mtcnn')
      if len(faces_features) > 1:
        print("Face not clear")
      for face_features in faces_features:
        input_features_with_face.append(face_features['embedding'])
        # print(face_features['embedding'])

    except Exception as e:
      print(f"Query {i+1} can't be detected face!", e)
      cant_detect_face_paths.append(image_path)

    if cant_detect_face_paths:
      if feature_without_face == 'CLIP':
        input_features_without_face = extract_clip_feature(cant_detect_face_paths, general=general)
      elif feature_without_face == 'BEiT':
        input_features_without_face = extract_beit_feature(cant_detect_face_paths, general=general)
      else:
        print("Feature not found")
    else:
      input_features_without_face = []

  # if len(input_features_without_face) > 0:
  #   if feature_without_face == 'CLIP':
  #     input_features_without_face = np.concatenate(input_features_without_face, axis=0)
  #   elif feature_without_face == 'BEiT':
  #     input_features_without_face = torch.cat(input_features_without_face, dim=0).numpy()

  return input_features_with_face, input_features_without_face

# Load database

In [ ]:
def get_database(film, feature_with_face, feature_without_face, general, remove_type):
  film_path = os.path.join(project_path, 'Group_Duy_Thang', 'index', film)
  if remove_type == '':
    database_with_face_path = os.path.join(film_path, 'With_Face', f'{feature_with_face}.pkl')
  else:
    database_with_face_path = os.path.join(film_path, 'With_Face', f'{feature_with_face}_{remove_type}.pkl')

  if remove_type == '':
    if general:
      database_without_face_path = os.path.join(film_path, 'Without_Face_All', f'{feature_without_face}_s.pkl' if feature_without_face == 'BEiT' else f'{feature_without_face}_rn.pkl')
    else:
      database_without_face_path = os.path.join(film_path, 'Without_Face_All', f'{feature_without_face}.pkl')
  else:
    if general:
      database_without_face_path = os.path.join(film_path, 'Without_Face_All', f'{feature_without_face}_s_{remove_type}.pkl' if feature_without_face == 'BEiT' else f'{feature_without_face}_rn_{remove_type}.pkl')
    else:
      database_without_face_path = os.path.join(film_path, 'Without_Face_All', f'{feature_without_face}_{remove_type}.pkl')

  print(database_with_face_path)
  print(database_without_face_path)

  with open(database_with_face_path, 'rb') as f:
    database_with_face = pickle.load(f)
    index_with_face = database_with_face["index"]
    map_index_with_face = database_with_face["map_index"]

  map_index_with_face = [filename.replace('.npy', '') for filename in map_index_with_face]
  # map_index_with_face = {i: value for i, value in enumerate(map_index_with_face)}
  # map_index_with_face[-1] = 'No shot'

  with open(database_without_face_path, 'rb') as f:
    database_without_face = pickle.load(f)
    index_without_face = database_without_face["index"]
    map_index_without_face = database_without_face["map_index"]

  map_index_without_face = [filename.replace('.npy', '') for filename in map_index_without_face]
  # map_index_without_face = {i: value for i, value in enumerate(map_index_without_face)}
  # map_index_without_face[-1] = 'No shot'

  return index_with_face, map_index_with_face, index_without_face, map_index_without_face

# Retrieval + Merge

In [ ]:
def retrieval(film_path, film, character, k, config):
  feature_with_face, feature_without_face, remove_type, general = config['feature_with_face'], config['feature_without_face'], config['remove_type'], config['general']
  print(f"Feature with face: {feature_with_face}")
  print(f"Feature without face: {feature_without_face}")
  print(f"Remove type: {remove_type}")
  print(f"General: {general}")

  character_path = os.path.join(film_path, character)
  print(character_path)
  list_query = [os.path.join(character_path, file) for file in os.listdir(character_path)]
  print(list_query)

  input_features_with_face, input_features_without_face = get_input_features(list_query, feature_with_face, feature_without_face, general=general)

  input_features_with_face = np.array(input_features_with_face)
  input_features_without_face = np.array(input_features_without_face)

  print(input_features_with_face.shape)
  print(input_features_without_face.shape)

  index_with_face, map_index_with_face, index_without_face, map_index_without_face = get_database(film, feature_with_face, feature_without_face, general, remove_type)

  # Search with face
  if input_features_with_face.shape[0] > 0:
    print("Search with face")
    D_with_face, I_with_face = index_with_face.search(normalize_data(input_features_with_face), k)     # actual search
    # print(I_with_face)                   # neighbors

    # Chuyển index thành tên shot
    results_with_face = []
    for query_results in I_with_face:
        # For each query, create a new list to store the mapped results
        mapped_results = []
        for result in query_results:
                mapped_results.append(map_index_with_face[result] if result > 0 else 'No shot')
        results_with_face.append(mapped_results)
    results_with_face = np.array(results_with_face)
    # print(results_with_face)
  else:
    D_with_face, I_with_face = None, None
    results_with_face = None


  # Search without face
  if input_features_without_face.shape[0] > 0:
    print("Search without face")                      # we want to see 4 nearest neighbors
    D_without_face, I_without_face = index_without_face.search(normalize_data(input_features_without_face), k)     # actual search
    # print(I_without_face)                   # neighbors

    # Chuyển index thành tên shot
    results_without_face = []
    for query_results in I_without_face:
        # For each query, create a new list to store the mapped results
        mapped_results = []
        for result in query_results:
                mapped_results.append(map_index_without_face[result] if result > 0 else 'No shot')
        results_without_face.append(mapped_results)
    results_without_face = np.array(results_without_face)
    # print(results_without_face)
  else:
    D_without_face, I_without_face = None, None
    results_without_face = None

  print("Search done")
  return results_with_face, results_without_face, D_with_face, D_without_face

In [ ]:
def merge_result(results_with_face, results_without_face, D_with_face, D_without_face, type='no'):
  if type == 'no':
    no_merge_result_1 = list(set(results_with_face.flatten())) if results_with_face is not None else []
    no_merge_result_2 = list(set(results_without_face.flatten())) if results_without_face is not None else []
    no_merge_result = no_merge_result_1 + no_merge_result_2
    no_merge_result = list(set(no_merge_result))
    no_merge_result = [x for x in no_merge_result if x != 'No shot']
    return no_merge_result

  elif type == 'Distance':
    if results_with_face is None and results_without_face is None:
      return []
    if results_with_face is None:
      results =  merge_sort_by_distance(results_without_face, D_without_face)
    elif results_without_face is None:
      results =  merge_sort_by_distance(results_with_face, D_with_face)
    else:
      results = merge_sort_by_distance(np.concatenate((results_with_face, results_without_face)), np.concatenate((D_with_face, D_without_face)))
    result = [x for x in results if x != 'No shot']
    return result

  elif type == 'Counter':
    if results_with_face is None and results_without_face is None:
      return []
    if results_with_face is None:
      results =  merge_sort_by_counter(results_without_face)
    elif results_without_face is None:
      results =  merge_sort_by_counter(results_with_face)
    else:
      results = merge_sort_by_counter(np.concatenate((results_with_face, results_without_face)))
    result = [x for x in results if x != 'No shot']
    return result

  elif type == 'Ranx':
    if results_with_face is None and results_without_face is None:
      return []
    method = 'rrf'
    if results_with_face is None:
      results =  merge_sort_by_ranx(results_without_face, D_without_face, method)
    elif results_without_face is None:
      results =  merge_sort_by_ranx(results_with_face, D_with_face, method)
    else:
      results = merge_sort_by_ranx(np.concatenate((results_with_face, results_without_face)), np.concatenate((D_with_face, D_without_face)), method)
    result = [x for x in results if x != 'No shot']
    return result

# Evaluate

In [ ]:
def get_map(character, k, config):
  list_ap = []
  for char in character:
    print(char)
    film = character[char]
    film_path = os.path.join(query_path, film)
    print(film_path)

    results_with_face, results_without_face, D_with_face, D_without_face = retrieval(film_path, film, char, k, config)
    result = merge_result(results_with_face, results_without_face, D_with_face, D_without_face, type=config['merge_type'])

    ground_truth = load_ground_truth(film)[char]

    # plot_precision_recall_curve(queries=[(ground_truth, result)], tittle='Precision-Recall Curve merge by distance')
    ap = average_precision(ground_truth, result)
    print(f"AP: {ap}")
    list_ap.append(ap)
  clear_output(wait=True)
  print(f'Mean Average Precision: {np.mean(list_ap)}')
  return np.mean(list_ap)

# Run

In [ ]:
character = {}

# Lặp qua các thư mục phim
for film_name in os.listdir(query_path):
    film_path = os.path.join(query_path, film_name)
    if os.path.isdir(film_path):
        # Lặp qua các thư mục con (tên nhân vật)
        for character_name in os.listdir(film_path):
            character_path = os.path.join(film_path, character_name)
            if os.path.isdir(character_path):
                character[character_name] = film_name

print(character)

{'Arvilla': 'bonneville', 'Bill': 'bonneville', 'Bo': 'bonneville', 'Carol': 'bonneville', 'Emmett': 'bonneville', 'Francine': 'bonneville', 'Margene': 'bonneville', 'Atsuko': 'little_rock', 'Brody': 'little_rock', 'Cory': 'little_rock', 'Dad': 'little_rock', 'Extra1': 'little_rock', 'Garbo': 'little_rock', 'Jordan': 'little_rock', 'Marques': 'little_rock', 'Rintaro': 'little_rock', 'Sean': 'little_rock', 'Francisco': 'little_rock', 'Burt': 'like_me', 'Gas_Station_Clerk': 'like_me', 'Homeless_Man': 'like_me', 'Kiya': 'like_me', 'Marshall': 'like_me', 'Young_Girl': 'like_me', 'Carlos': 'losing_ground', 'Celia': 'losing_ground', 'Duke': 'losing_ground', 'Extra9': 'losing_ground', 'George': 'losing_ground', 'Gypsy': 'losing_ground', 'Mother': 'losing_ground', 'Sara': 'losing_ground', 'Victor': 'losing_ground', 'Extra5': 'ChainedforLife', 'Herr_Director': 'ChainedforLife', 'Mabel': 'ChainedforLife', 'Max': 'ChainedforLife', 'Michelle': 'ChainedforLife', 'Rosenthal': 'ChainedforLife', 'Sara

In [ ]:
import time
results = pd.DataFrame(columns=['Feature with face', 'Feature without face', 'Remove type', 'General', 'Merge type', 'MAP', 'Execute time'])
for i, config in enumerate(param_grid):
    if i < 24:
      print(f"Tổ hợp {i+1}: {config}")
      start_time = time.time()
      map = get_map(character, k, config)
      end_time = time.time()
      print(f"Time: {end_time - start_time}s")
      print('-'*30)
      # save to dataframe
      results.loc[i] = [config['feature_with_face'], config['feature_without_face'], config['remove_type'], config['general'], config['merge_type'], map, end_time - start_time]

# save
results.to_excel(os.path.join(project_path, "Group_Duy_Thang", "result_1.xlsx"))



Mean Average Precision: 0.24583875682862336
Time: 397.1449565887451s
------------------------------


In [ ]:
results

,Feature with face,Feature without face,Remove type,General,Merge type,MAP,Execute time
0,Facenet,BEiT,Compare,True,Counter,0.218624,707.993183
1,Facenet,BEiT,Compare,True,Distance,0.293213,487.856769
2,Facenet,BEiT,Compare,True,Ranx,0.231589,748.187961
3,Facenet,BEiT,Compare,False,Counter,0.218210,495.102886
4,Facenet,BEiT,Compare,False,Distance,0.285293,493.212195
5,Facenet,BEiT,Compare,False,Ranx,0.230470,474.951481
6,Facenet,BEiT,DBSCAN,True,Counter,0.211818,508.984989
7,Facenet,BEiT,DBSCAN,True,Distance,0.329620,485.523080
8,Facenet,BEiT,DBSCAN,True,Ranx,0.241912,475.891390
9,Facenet,BEiT,DBSCAN,False,Counter,0.211464,496.039658
